In [1]:
from  metadata.metadata import _search_data_catalog,_get_entry_details,get_dependencies_info,_get_table_profile,generate_table_description


#"bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg"
entry_id="//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/ccagg"
result = _get_entry_details(entry_id)  # Replace this with your function to get a result by ID
additional_info=get_dependencies_info(result['fullyQualifiedName'])
profile_info=_get_table_profile(entry_id)


/Users/jskuratowicz/Projects/Metadata Generation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d", "source": {"fullyQualifiedName": "bigquery:bigquery-public-data.chicago_crime.crime"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.ccagg"}, "startTime": "2024-02-05T11:08:23.637Z", "endTime": "2024-02-05T11:08:35.671Z"}]}
find_lineage looking for link:projects/446454295341/locations/us/links/p:0116a1f484c3479853a431f90f6a696d
get_dependencies_info:found process through lineage: projects/446454295341/locations/us/processes/5c66d1d1e5c06e9a6e405e63cbad2378
US:scri

In [6]:
import time
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

def _get_prompt(tablename,profile,sql_queries):
    queries=""
    for i in sql_queries:
        queries=queries+i.query+";"

    contents="You are a data steward. Your role is to produce human meaningful metadata descriptions.\
    Table's fully qualified "+tablename+" \
    These SQL queries can be used to generate the data in table \
    "+ queries +"\
    This is table profile information "+profile+"\
    Provide  description of table contents and usage. \
    Provide description of each column with top common values and their explanation. \
    Provide pseudocode expressed in SQL to calculate each field formatted in LaTeX.\
    "
    return contents

def generate_table_description(tablename,profile,sql_queries):

    prompt=_get_prompt(tablename,profile,sql_queries)
    text_model= GenerativeModel("gemini-ultra")
    responses = text_model.generate_content(prompt)
    print("\n-------Response--------")
#   for response in responses:
 #       print(response.text, end="")
    return responses

generated_description=generate_table_description(tablename=result['fullyQualifiedName'],profile=profile_info,\
                                                     sql_queries=additional_info)


-------Response--------


In [21]:
returned_text=""
for i in generated_description.candidates:
    print(i.content.parts)
    for j in i.content.parts:
        returned_text+=j.text

print(returned_text)


[text: "**Description of Table Contents and Usage**\n\nThe `ccagg` table contains aggregated crime data from the city of Chicago. The data includes the number of crimes, categorized by various factors such as the type of crime, time of occurrence, and location. This table can be used to analyze crime trends, identify patterns, and inform decision-making related to public safety.\n\n**Description of Columns**\n\n| Column | Data Type | Description | Top Common Values and Explanation |\n|---|---|---|---|\n| `cuk` | INTEGER | Count of unique crime incidents | 1: Majority of incidents have only one crime reported |\n| `m` | INTEGER | Month of crime occurrence | 7 (July): Peak crime season during summer months |\n| `d` | INTEGER | Day of crime occurrence | 15: Mid-month days tend to have higher crime rates |\n| `h` | INTEGER | Hour of crime occurrence | 12 (Noon): Midday hours see higher crime activity |\n| `pt` | STRING | Primary type of crime | THEFT: Most common crime type in Chicago |\n|